# Configuration Space

In [ ]:
#|default_exp confspace
#|export
from typing import List, Set, Dict, Tuple, Optional
import yaml
import numpy as np
import json
from nbx.fileio import dump, load
from pathlib import Path
from nbx.utils import Bunch


def is_lambda_str(val):
    return type(val) == str and val[:6] == "lambda"


def is_basically_a_function(val):
    return hasattr(val, '__call__') or is_lambda_str(val)
               

def is_basically_a_list(val):
    return hasattr(val, '__iter__') and type(val) != str


def replace_maybe(k, abbr):
    if k in abbr: return abbr[k]
    else: return k



## Single Config

In [ ]:
#| export
class Conf(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(**kwargs)
        args = list(args)
        if len(args) == 0: args.append(-1)
        if len(args) == 1: args.append(None)
        self.t, self.source = args[:2]
        
    @property
    def index(self):
        return self.t
    
    def bunch(self):
        return Bunch(**self)
    
    
c = Conf(1, "unknown", **{"x": 1})
c.index, c.source, c.bunch(), c

(1, 'unknown', {'x': 1}, {'x': 1})

## Configuration Space

In [ ]:
#| export
class ConfSpace(object):
    def __init__(self, fname="", order="col"):
        self.set_order(order)
        self.fname  = fname
        self.shape  = ()
        self.keys   = []
        self.vals   = []
        self.abbr   = {}
        self.attach = [] # attach to label


    def add(self, key, vals, abbr=None, attach=False):
        """
        Add a coordinate axis with given values to the configuration space.

        You can also add Axes "labeled" with lamda functions (instead of a list of values). 
        The function is called with no arguments but the config object as its global context, 
        to compute the corresponding value.
        """
        if key in self.keys: 
            raise Exception(f"Key `{key}` already in there....") 
        if not (is_basically_a_list(vals) or is_lambda_str(vals)): 
            raise Exception(f"Values `{vals}` should be iterable or a special lambda str...")


        if is_lambda_str(vals): 
            self.shape += (1,)
        else:
            vals = list(vals)
            self.shape += (len(vals),)

        self.keys.append(key)
        self.vals.append(vals)
        if attach: self.attach.append(key)
        if abbr is None or abbr == "": abbr = key.split(":")[-1]
        self.abbr[key]= abbr
    
        return self


    def __getitem__(self, i):
        if type(i) == int:
            assert i < len(self), f"{i} = key >= length = {len(self)}"
            return self.get_config(i)
        else:
            raise Exception("Get can only handle integer keys atm...")


    def get_config(self, t):
        p = Path(self.fname)        
        conf = Conf(t, self.fname)



        func_items = []
        I = self.get_multi_index(t)
        for k, i in enumerate(I):

            key   = self.keys[k]
            vals  = self.vals[k] 

            if is_basically_a_list(vals):
                conf[key] = vals[i]
            else:
                # Handling dynamic Values later
                func_items.append((key,vals))


        # Finally handling dynamic Values
        for k,f in func_items:
            if is_lambda_str(f):
                v = eval(f"({f})()", conf.bunch())
                conf[k] = v
            else: raise Exception("Can't handle `{f}`")

        return conf


    def filter(self, constr):
        """
            Returns a list of pairs (t, c)
        """
        if hasattr(constr, '__call__'):
            return [(t,c) for t,c in enumerate(self) if constr(Bunch(**c))==True]
        else:

            return filter_confspace(constr, self)


    def __len__(self):
        if len(self.shape)>0: return np.product(self.shape)
        else: return 0

    
    def __iter__(self):
        self._i = 0
        return self


    def __next__(self):
        if self._i < len(self):
            p = self[self._i]
            self._i += 1
            return p
        else:
            raise StopIteration


    @property
    def dim(self):
        return len(self.shape)


    def items(self):
        return zip(self.keys, self.vals)


    def get_multi_index(self, t):
        """
        “Row-major” ordering is also referred to as “C-ordering” 
        because this is the traversal method utilized in the C language. 
        “Column-major” ordering, on the other hand, is also referred to as “F-ordering”, 
        because it is used by the Fortran language.
        """
        shape = self.shape
        order = self.order
        n = len(shape)
        I = [None]*n
        for k in range(0,n):
            if order == "row": 
                i = t//np.prod(shape[k+1:])
                I[k] = int(i)%shape[k]
            if order == "col":
                i = t//np.prod(shape[:k])
                I[k] = int(i)%shape[k]

        return I

    def coords(self, c):
        I = [None]*len(self.shape)
        assert isinstance(c, dict)

        for k,v in get_items(c):
            j = self.keys.index(k)
            vals = self.vals[j]

            if is_lambda_str(vals):
                if v == eval(f"({vals})()", bunch(c)):
                    I[j] = 0
                else:
                    raise Exception(f"can't find this config in here: {key}={v}")
            else:
                if vals.index(v) >=0:
                    I[j] = vals.index(v)
                else:
                    raise Exception(f"can't find this config in here: {key}={v}")

        return I


    def find(self, c):
        assert isinstance(c, dict)
        I = self.coords(c)

        t = 0
        for j,i in enumerate(I):
            t += i*np.prod(self.shape[j+1:])

        return int(t)


    def show_2dgrid(self, k=None, l=None):
        k,l = self.keys[:2]
        i,j = self.keys.index(k), self.keys.index(l)
        a,b = self.abbr[k], self.abbr[l]
        

        grid = np.stack(np.meshgrid(self.vals[j], self.vals[i]),axis=2)
        print(f"--- ({a} {b}):\n")
        for i,row in enumerate(grid):
            s = '  '.join(["{:10s}".format(str(c)[:10]) for c in row])
            print(s)
        print("\n---")


    def __str__(self):
        s = f"ConfSpace({{\n  |confs| = {len(self)}\n"
        if self.fname is not None: 
                s = f"{self.fname}:ConfSpace({{\n  |confs| = {len(self)}\n"
        items = []
        for k,v in zip(self.keys, self.vals):
            k_ =f"'{k}'"
            items.append(f"  {k_}\t : {v}")
        s += ",\n".join(items)
        s += "\n})"
        return s


    def __repr__(self):
        return str(self)


    def sample(self):
        t = np.random.choice(len(self))
        x = self[t]
        return x


    def set_order(self, order):
        """
        Sets the order of traversal of the parameter array.
        Note that the first dimension is the "column" (col) direction,
        and the second dimension is the "row" direction.
        """
        assert order == "col" or order == "row"
        self.order = order


    def dump(self, fname=None):
        assert fname is not None or self.fname is not None

        if fname is None: 
            fname = self.fname


        dump({
            'state': {
                'len': len(self),
                'attach': self.attach,
                'abbr': self.abbr,
                'keys': self.keys}, 
            'dictitems': dict(self.items())
        }, fname)

        return fname


    def load(self, fname):
        d = load(fname)
        self.fname= fname
        self.from_loaded(d)

        return self


    def from_loaded(self, d):
        assert has_key(d, 'state') and has_key(d, 'dictitems')


        if 'keys' in d['state']:
            items = [(k, get_val(d['dictitems'], k)) for k in d['state']['keys']]
        else:
            items = get_items(d['dictitems'])


        for k,vals in items:


            # This enables to load a single config 
            # as a 1-element space
            if not is_basically_a_function(vals):
                if not hasattr(vals, '__iter__') or type(vals) == str:
                    vals = [vals]
            else:
                if not is_lambda_str(vals):
                    raise Exception("can only handle special lambda strings atm")

            self.add(k, vals)

        self.attach = d['state']['attach']
        self.abbr   = d['state']['abbr']

    
    
def is_iter(x):
    return hasattr(x, "__iter__") and not isinstance(x, str)

def wrap(x):
    if not is_iter(x): return [x]
    else: return x
    
def filter_confspace(constr:Dict, confspace:ConfSpace) -> List[Tuple[int, dict]]:
    return [(t,c) for t,c in enumerate(confspace)
            if all( [c[k] in wrap(v) for k,v in constr.items()])]



In [ ]:
C = ConfSpace("_test_space.yaml")
C.add("x", range(10), attach=True)
C.add("y", ["a", "b"], attach=False)

In [ ]:
C[0]

In [ ]:
C.dump()

In [ ]:
!open _test_space.yaml

In [ ]:
!ls

In [ ]:
C[0].index, C[0].source